<a href="https://colab.research.google.com/github/kikun1234/Medium_Python_Stats/blob/main/Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##importing libraries & import google sheets
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#stats
ws = gc.open('Stats').get_worksheet(1)
df = pd.DataFrame(ws.get_all_records())
df1 = df[~df['Title'].str.contains('May 2021|February 2022|March 2022|April 2022')]
df1['Title'] = df1['Title'].shift(1)
df2 = df1.reset_index(drop=True)
df2 = df2.iloc[1:]
df2= df2[~df2['Title'].str.contains('storyDetails')]

def clean(x):
  df4 = x.str.split('([^\W\d])', expand=True)
  df4 = df4.iloc[: , :-1]
  df4.columns=["A","B"]
  df4['B'] = df4['B'].str.replace("K","1000")
  df4['A'] = df4['A'].astype('float64')
  df4['B'] = df4['B'].astype('float64')
  df4['C'] = df4['A']*df4['B']
  return(df4['C'])

df3 = df2
views1 = clean(df2['Views'])
df3["Views1"] = views1
df3['Views1'] = df3['Views1'].fillna(df3['Views'])

reads1 = clean(df2['Reads'])
df3["Reads1"] = reads1
df3['Reads1'] = df3['Reads1'].fillna(df3['Reads'])

df3['Read Ratio']=df3['Read Ratio'].str.replace('\%','')
df3['Read Ratio'] = df3['Read Ratio'].astype('float64')
df3['Read Ratio'] = df3['Read Ratio']/100

df4=df3
df4['Views2']=df4['Reads1']/df4['Read Ratio']
df4['Views2'] = df4['Views2'].round(0)
df4['Fans'] = df4['Fans'].astype('float64')
df_stats1 = df4[['Title','Views2','Reads1','Read Ratio','Fans']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
#your@ page
ws = gc.open('Stats').get_worksheet(0)
df = pd.DataFrame(ws.get_all_records())
df['Title'] = df['Title'].replace('', np.nan) 
df1 = df.dropna()
df1 = df1[df1['Title'].str.len()<100]
df1 = df1[~df1['Title'].str.contains('Published in|Get an email whenever Jason Huynh publishes.|You cannot subscribe to yourself')]
df2 = df1
df2 = df2[df2['Title'].str.len()>20]
titles = df2.iloc[::2].reset_index(drop=True)
df4 = df1[df1['Title'].str.len()<20]
date = df4[::3].reset_index(drop=True)
tag = df4[1::3].reset_index(drop=True)
time = df4[2::3].reset_index(drop=True)
df_stats2 = pd.concat([titles,date, tag, time],axis=1)
df_stats2.columns = ['Title','Date', 'Tag', 'Time']
df_stats2['Date'] = df_stats2['Date'].str.replace('\·','')
df_stats2['Time'] = df_stats2['Time'].str.replace('min read','')
df_stats2['Time'] = df_stats2['Time'].astype("float64")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
#earnings
ws = gc.open('Stats').get_worksheet(2)
df = pd.DataFrame(ws.get_all_records())
df1 = df[~df['Title'].str.contains("• View stats")]
df2 = df1[::2].reset_index(drop=True)
df3 = df1[1::2].reset_index(drop=True)
df_stats3 = pd.concat([df2,df3],axis=1)
df_stats3.columns = ['Title','Earnings']
df_stats3['Earnings']=df_stats3['Earnings'].str.replace("\$","")
df_stats3['Earnings']=df_stats3['Earnings'].astype("float64")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [ ]:
df_final = df_stats1.merge(df_stats2,on="Title")
df_final = df_final.merge(df_stats3,how="left",on="Title")

In [ ]:
#stats function
df_final.loc[df_final['Tag']=="Entrepreneurship","Topic"]="Startup"
df_final.loc[df_final['Tag']=="Startup","Topic"]="Startup"
df_final.loc[df_final['Tag']=="Passive Income","Topic"]="Startup"

df_final.loc[df_final['Tag']=="Cryptocurrency","Topic"]="Blockchain"
df_final.loc[df_final['Tag']=="Blockchain","Topic"]="Blockchain"
df_final.loc[df_final['Tag']=="Nft","Topic"]="Blockchain"
df_final.loc[df_final['Tag']=="Metaverse","Topic"]="Blockchain"

df_final.loc[df_final['Tag']=="Data Science","Topic"]="Data Science"
df_final.loc[df_final['Tag']=="Machine Learning","Topic"]="Data Science"
df_final.loc[df_final['Tag']=="Data","Topic"]="Data Science"
df_final.loc[df_final['Tag']=="Spreadsheets","Topic"]="Data Science"

df_final.loc[df_final['Tag']=="Data Lake","Topic"]="Programming"
df_final.loc[df_final['Tag']=="Programming","Topic"]="Programming"
df_final.loc[df_final['Tag']=="Python","Topic"]="Programming"

df_final.loc[df_final['Tag']=="Business","Topic"]="Investing"
df_final.loc[df_final['Tag']=="Investing","Topic"]="Investing"
df_final.loc[df_final['Tag']=="Money","Topic"]="Investing"
df_final.loc[df_final['Tag']=="Personal Finance","Topic"]="Investing"
df_final.loc[df_final['Tag']=="Inflation","Topic"]="Investing"

df_final.loc[df_final['Tag']=="Work","Topic"]="Work"
df_final.loc[df_final['Tag']=="Technology","Topic"]="Work"

df_final.loc[df_final['Tag']=="Self Improvement","Topic"]="Self Improvement"
df_final.loc[df_final['Tag']=="Problem Solving","Topic"]="Self Improvement"
df_final.loc[df_final['Tag']=="Life Lessons","Topic"]="Self Improvement"
df_final.loc[df_final['Tag']=="Writing","Topic"]="Self Improvement"

df_final1 = df_final[~df_final['Topic'].isna()]

def stat_clean(x):
  df_final2 = df_final1.sort_values(by=[x],ascending=False)
  top = int(round(len(df_final2)*0.2,0))
  df_final2_top = df_final2.head(top)
  df_final2_top['Top'] = "Top"
  df_final3 = pd.concat([df_final2,df_final2_top['Top']],axis=1).sort_values(by=[x],ascending=False)
  stats = df_final3[['Topic','Top']]
  stats1 = stats.groupby('Topic').count().reset_index()
  stats_totals = pd.DataFrame(stats['Topic'].value_counts())
  stats_totals.columns=['Total']
  stats_totals = stats_totals.reset_index()
  stats_totals.columns=['Topic','Total']
  stats2 = stats1.merge(stats_totals,on="Topic")
  stats2['Top+1']=stats2['Top']+1
  stats2['Total+2']=stats2['Total']+2
  stats2['%Success']=stats2['Top+1']/stats2['Total+2']
  return(stats2)

In [ ]:
ws = gc.open('Output').get_worksheet(0)
df_final_update = df_final.fillna(0)
ws.update([df_final_update.columns.values.tolist()] + df_final_update.values.tolist())

{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 450,
 'updatedColumns': 9,
 'updatedRange': 'All!A1:I50',
 'updatedRows': 50}

In [ ]:
views = stat_clean('Views2')
ws = gc.open('Output').get_worksheet(1)
ws.update([views.columns.values.tolist()] +views.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 48,
 'updatedColumns': 6,
 'updatedRange': 'Views!A1:F8',
 'updatedRows': 8}

In [ ]:
reads = stat_clean('Reads1')
ws = gc.open('Output').get_worksheet(2)
ws.update([reads.columns.values.tolist()] +reads.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 48,
 'updatedColumns': 6,
 'updatedRange': 'Reads!A1:F8',
 'updatedRows': 8}

In [ ]:
ratio = stat_clean('Read Ratio')
ws = gc.open('Output').get_worksheet(3)
ws.update([ratio.columns.values.tolist()] +ratio.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 48,
 'updatedColumns': 6,
 'updatedRange': 'Ratio!A1:F8',
 'updatedRows': 8}

In [ ]:
fans = stat_clean('Fans')
ws = gc.open('Output').get_worksheet(4)
ws.update([fans.columns.values.tolist()] +fans.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 48,
 'updatedColumns': 6,
 'updatedRange': 'Fans!A1:F8',
 'updatedRows': 8}

In [ ]:
earnings = stat_clean('Earnings')
ws = gc.open('Output').get_worksheet(5)
ws.update([earnings.columns.values.tolist()] +earnings.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'spreadsheetId': '1vSP_CuDba56tHC92gRueIR0TeZ0NrIGu7flhWadPUHg',
 'updatedCells': 48,
 'updatedColumns': 6,
 'updatedRange': 'Earnings!A1:F8',
 'updatedRows': 8}